# Chapter 5. Object References, Mutability, and Recycling
---

## ToC

1. [Copies Are Shallow by Default](#copies-are-shallow-by-default)  
    1.1. [Deep and Shallow Copies of Arbitrary Objects](#deep-and-shallow-copies-of-arbitrary-objects)  
    1.2. [Summary](#summary)
2. [Function Parameters as References](#function-parameters-as-references)  
    2.1. [Mutable Types as Parameter Defaults: Bad Idea](#mutable-types-as-parameter-defaults-bad-idea)  
    2.2. [Defensive Programming with Mutable Parameters](#defensive-programming-with-mutable-parameters)
---

## Copies Are Shallow by Default

The easiest way to copy a list (or most built-in mutable collections) is to use the builtin
constructor for the type itself.

**I.** Use Constructor

In [1]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)
l2

[3, [55, 44], (7, 8, 9)]

In [2]:
# the copies are equal
l2 == l1

True

In [3]:
# but refer to two different objects
l2 is l1

False

**II.** Use Shortcut `[:]`

In [1]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = l1[:]
# the copies are equal
l2 == l1

True

In [2]:
# but refer to two different objects
l2 is l1

False

**III.** Use `=`  

In [8]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = l1
l2 == l1

True

In [9]:
l2 is l1

True

For lists and other mutable sequences, the shortcut `l2 = l1[:]` also makes a copy.

However, using the constructor or `[:]` produces a *shallow copy* (i.e., the outermost
container is duplicated, but the copy is filled with references to the same items held
by the original container). This saves memory and causes no problems if all the items
are immutable. But if there are mutable items, this may lead to unpleasant surprises.

**Example:** create a shallow copy of a list containing another list and a tuple,
and then make changes to see how they affect the referenced objects.

In [5]:
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = l1
l1.append(100)
l1[1].remove(55)
print('l1:', l1)
print('l2:', l2)

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9), 100]


![Figure 90](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/90.PNG)

In [8]:
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)
l1.append(100)
l1[1].remove(55)
print('l1:', l1)
print('l2:', l2)

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]


![Figure 91](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/91.PNG)

In [10]:
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)
l2[1] += [33, 22]
l2[2] += (10, 11)
print('l1:', l1)
print('l2:', l2)

l1: [3, [66, 55, 44, 33, 22], (7, 8, 9)]
l2: [3, [66, 55, 44, 33, 22], (7, 8, 9, 10, 11)]


For a mutable object like the list referred by `l2[1]`, the operator `+=` changes the
list in place. This change is visible at `l1[1]`, which is an alias for `l2[1]`.


`+=` on a tuple creates a new tuple and rebinds the variable `l2[2]` here. This is the
same as doing `l2[2] = l2[2] + (10, 11)`. Now the tuples in the last position of
`l1` and `l2` are no longer the same object.

![Figure 92](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/92.PNG)

![Figure 93](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/93.PNG)

### Deep and Shallow Copies of Arbitrary Objects

The `copy` module provides the `deepcopy` and `copy` functions that return deep and
shallow copies of arbitrary objects.

Concise but informative Youtube video: [Shallow and Deep Copy Python Programming Tutorial](https://www.youtube.com/watch?v=naG4uXpmVAU)

**Example.** Bus picks up and drops off passengers

In [22]:
class Bus:

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [2]:
import copy
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
bus4 = bus1
id(bus1), id(bus2), id(bus3), id(bus4)

(1937597100352, 1937597210912, 1937597206832, 1937597100352)

After `bus1` drops `'Bill'`, he is also missing from `bus2`.

In [3]:
bus1.drop('Bill')
bus2.passengers

['Alice', 'Claire', 'David']

In [4]:
id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)

(1937597355456, 1937597355456, 1937597379776)

`bus3` is a deep copy of `bus1`, so its `passengers` attribute refers to another list.

In [5]:
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

In [6]:
a = [10, 20]
b = [a, 30]

Note that making deep copies is not a simple matter in the general case. Objects may
have cyclic references that would cause a naïve algorithm to enter an infinite loop.
The `deepcopy` function remembers the objects already copied to handle cyclic references
gracefully, as shown in following example:

In [6]:
from copy import deepcopy
a = [10, 20]
b = [a, 30]
a.append(b)
a


[10, 20, [[...], 30]]

In [7]:
a[2]

[[10, 20, [...]], 30]

In [8]:
c = deepcopy(a)
c

[10, 20, [[...], 30]]

#### Summary

```python
import copy

L1 = ['a', ['b', 'c'], ('d', 'e')]
L2 = L1
L3 = list(L1)
L4 = copy.copy(L1)
L5 = copy.deepcopy(L1)
```
---
| Variable | Type of Copy         | `L1 is ...` | `L1[1] is ...` | Changes affect `L1`? | Implementation      | Description                                                                 |
|----------|----------------------|-------------|----------------|-----------------------|---------------------|-----------------------------------------------------------------------------|
| `L2`     | Assignment (alias)   | ✅ Same      | ✅ Same         | ✅ Yes (fully linked) | `L2 = L1`           | L2 is L1 — they reference the same object. `==` and `IS` both yield same.                                |
| `L3`     | Shallow via `list()` | ❌ New list  | ✅ Same         | ⚠️ Yes (inner shared) | `L3 = list(L1)`     | L3 is a new list object, however, its elements are references to the same objects inside L1. |
| `L4`     | Shallow via `copy`   | ❌ New list  | ✅ Same         | ⚠️ Yes (inner shared) | `L4 = copy(L1)`     | Similar to L3, it's a new outer list, but elements are not copied recursively. So inner list and tuple still reference the same objects. |
| `L5`     | Deep via `deepcopy`  | ❌ New list  | ❌ New          | ❌ No (fully independent) | `L5 = deepcopy(L1)` | Entire structure is copied recursively. All nested objects are new and independent.         |


In behavior, `L3 = list(L1)` and `L4 = copy.copy(L1)` are almost identical because both create shallow copies of the outer list. However, the difference lies in how the copy is made — the constructor vs the copy protocol — which can matter for subclasses of list.

In [ ]:
import copy

class MyList(list):
    def __init__(self, *args):
        super().__init__(*args)
        self.metadata = "custom"

L1 = MyList(['a', ['b', 'c'], ('d', 'e')])
L3 = list(L1)           # Shallow copy using constructor
L4 = copy.copy(L1)      # Shallow copy using copy protocol

print(type(L1))  # <class '__main__.MyList'>
print(type(L3))  # <class 'list'>               ← lost the custom subclass
print(type(L4))  # <class '__main__.MyList'>   ← preserved the subclass

print(hasattr(L3, 'metadata'))  # False
print(hasattr(L4, 'metadata'))  # True


<class '__main__.MyList'>
<class 'list'>
<class '__main__.MyList'>
False
True


### Function Parameters as References

The only mode of parameter passing in Python is *call by sharing*. That is the same
mode used in most object-oriented languages, including JavaScript, Ruby, and Java. Call by sharing
means that each formal parameter of the function gets a copy of each reference in the
arguments. In other words, the parameters inside the function become aliases of the
actual arguments.

The result of this scheme is that a function may change any mutable object passed as
a parameter, but it cannot change the identity of those objects (i.e., it cannot altogether
replace an object with another).

In [15]:
def f(a, b):
    a += b
    return a

**Ex. Number:** Unchanged

In [ ]:
x = 1
y = 2
f(x, y)

In [12]:
x, y

(1, 2)

**Ex. List:** Changed

In [13]:
a = [1, 2]
b = [3, 4]
f(a, b)

[1, 2, 3, 4]

In [19]:
a, b

([1, 2, 3, 4], [3, 4])

**Ex. Tuple:** Unchanged

In [16]:
t = (10, 20)
u = (30, 40)
f(t, u)

(10, 20, 30, 40)

In [18]:
t, u

((10, 20), (30, 40))

#### Mutable Types as Parameter Defaults: Bad Idea 

In following example, we tried to be clever, and instead of having a default value of `passengers=None`, we have `passengers=[]`, thus avoiding the if in the previous `__init__`. This “cleverness” gets us into trouble.

**Example:** illustrate the danger of a mutable default

In [1]:
class HauntedBus:
    """A bus model haunted by ghost passengers"""

    def __init__(self, passengers=[]):
        self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [2]:
bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers

['Alice', 'Bill']

In [3]:
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers

['Bill', 'Charlie']

In [4]:
bus2 = HauntedBus()
bus2.pick('Carrie')
bus2.passengers

['Carrie']

In [5]:
bus3 = HauntedBus()
bus3.passengers

['Carrie']

In [6]:
bus3.pick('Dave')
bus3.passengers

['Carrie', 'Dave']

In [7]:
bus2.passengers is bus3.passengers

True

In [9]:
bus1.passengers

['Bill', 'Charlie']

**The problem:** `bus2.passengers` and `bus3.passengers` refer to the same list.  

But bus1.passengers is a distinct list.

#### Defensive Programming with Mutable Parameters

When you are coding a function that receives a mutable parameter, you should carefully
consider whether the caller expects the argument passed to be changed.

For example, if your function receives a dict and needs to modify it while processing
it, should this side effect be visible outside of the function or not? Actually it depends
on the context. It’s really a matter of aligning the expectation of the coder of the function
and that of the caller.

The last bus example in this chapter shows how a `TwilightBus` breaks expectations
by sharing its passenger list with its clients. Before studying the implementation, see how the `TwilightBus` class works from the perspective of a client of the class:

```python
>>> basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
>>> bus = TwilightBus(basketball_team)
>>> bus.drop('Tina')
>>> bus.drop('Pat')
>>> basketball_team
['Sue', 'Maya', 'Diana']
```

`TwilightBus` violates the [“Principle of least astonishment,”](https://deviq.com/principles/principle-of-least-astonishment) a best practice of interface
design.3 It surely is astonishing that when the bus drops a student, their name is
removed from the basketball team roster.

In [11]:
class TwilightBus:
    """A bus model that makes passengers vanish"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

Reminding original `Bus` class:

```python
class Bus:

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)
```

What's different is:
- `Bus`: `self.passengers = list(passengers)` -> Shallow Copy 
- `HauntedBus`: `self.passengers = passengers` -> Assignment (alias)

In `Bus` the argument passed to the passengers parameter may be a tuple or any other iterable, like a set or even database results, because the list constructor accepts any iterable.

![Figure 94](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/94.PNG)